In [2]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import bz2
from functools import partial
from collections import Counter
import pickle
from itertools import islice
from xml.etree import ElementTree
import codecs
import csv
import time
import os
import re
import gdown
from pathlib import Path
from google.cloud import storage

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
    for line in f:
        parts = line.split(' ')
        wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
    pickle.dump(wid2pv, f)


--2023-01-09 23:47:32--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.75MB/s    in 8m 36s  

2023-01-09 23:56:08 (4.63 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [ ]:
# read in the counter
with open(pv_clean, 'rb') as f:
    wid2pv = pickle.loads(f.read())

In [ ]:
def pageviews(wiki_ids):
        res = []
        for article_id in wiki_ids:
            res.append(wid2pv[article_id])
        return res

In [ ]:
pageviews([10430, 17769])

[2077, 5713]

In [13]:
bucket_name = "elad_318640828"
# write out the counter as binary file (pickle it)

p = pickle.dumps(wid2pv)
# Create a client for interacting with GCS
gcs_client = storage.Client()

# Get a reference to the bucket
bucket = gcs_client.bucket(bucket_name)

# Create a new blob in the bucket and upload the pickle file
blob = bucket.blob(f'wid2pv.pkl')
blob.upload_from_string(p, content_type='application/octet-stream')